# day 17

# day 16

# day 15

# day 14

# day 13

My first attempt at Part Two was even slower, as it repeated the full timestep sequence for every delay duration calculation. Might have worked if I gave it a week to run ...

In [1]:
from itertools import cycle


def make_scanner(rng):
    r = list(range(rng))
    return cycle(r + r[-2:0:-1])


def make_firewall(spec):
    fw = []
    max_depth = max(spec.keys())
    for d in range(max_depth + 1):
        r = spec.get(d, 0)
        if r:
            fw.append(make_scanner(r))
        else:
            fw.append(cycle([-1]))
    return fw


def make_fw_state_table(spec, table_len):
    fw = make_firewall(spec)
    fw_state_table = []
    for _ in range(table_len):
        fw_state_table.append([next(s) for s in fw])
    return fw_state_table


def make_traverse(fw_spec, delay):
    fw = make_firewall(fw_spec)
    for _ in range(delay):
        fw_sp = [next(s) for s in fw]
    severity = 0
    for tp in range(max(fw_spec.keys())+1):
        fw_sp = [next(s) for s in fw]
        if fw_sp[tp] == 0:
            severity += tp*fw_spec[tp]
    return severity

def make_traverse_w_state_table(fw_spec, fw_state_table, delay):
    for tp in range(max(fw_spec.keys())+1):
        if fw_state_table[tp+delay][tp] == 0:
            return False
    return True

In [2]:
test_input = """0: 3
1: 2
4: 4
6: 4"""

test_fw_spec = {}
for line in test_input.split('\n'):
    dep, rng = [int(n) for n in line.split(': ')]
    test_fw_spec[dep] = rng

In [3]:
fw_spec = {}
with open('input_day_13.txt') as f:
    for line in f:
            dep, rng = [int(n) for n in line.split(': ')]
            fw_spec[dep] = rng

In [4]:
[(d, make_traverse(test_fw_spec, d)) for d in range(12)]

[(0, 24),
 (1, 2),
 (2, 16),
 (3, 2),
 (4, 0),
 (5, 2),
 (6, 24),
 (7, 2),
 (8, 16),
 (9, 2),
 (10, 0),
 (11, 2)]

In [5]:
fw_state_table = make_fw_state_table(test_fw_spec, 1000)
[(d, make_traverse_w_state_table(test_fw_spec, fw_state_table, d)) for d in range(12)]

[(0, False),
 (1, False),
 (2, False),
 (3, False),
 (4, False),
 (5, False),
 (6, False),
 (7, False),
 (8, False),
 (9, False),
 (10, True),
 (11, False)]

In [6]:
make_traverse(fw_spec, 0)

1900

In [7]:
%%time
fw_state_table = make_fw_state_table(fw_spec, 10000000)

CPU times: user 3min 12s, sys: 53.2 s, total: 4min 5s
Wall time: 4min 16s


In [8]:
%%time
d = 0
while not make_traverse_w_state_table(fw_spec, fw_state_table, d):
    d += 1

print(d)

3966414
CPU times: user 10.7 s, sys: 4.32 s, total: 15 s
Wall time: 15.5 s


In [9]:
%%time
# and now the quick way ...
depths_cycles = [(k, 2*v-2) for k, v in fw_spec.items()]

def caught_at_delay(depths_cycles, delay):
    return 0 in [(d + delay) % c for d, c in depths_cycles]

d = 0
while caught_at_delay(depths_cycles, d):
    d += 1
print(d)

3966414
CPU times: user 18.5 s, sys: 31.9 ms, total: 18.5 s
Wall time: 18.6 s


In [11]:
%%time
# maybe even quicker ...
depths_cycles = [(k, 2*v-2) for k, v in fw_spec.items()]

def caught_at_delay(depths_cycles, delay):
    for d, c in depths_cycles:
        if (d + delay) % c == 0:
            return True
    return False

d = 0
while caught_at_delay(depths_cycles, d):
    d += 1
print(d)

3966414
CPU times: user 2.07 s, sys: 2.05 ms, total: 2.07 s
Wall time: 2.07 s


# day 12

In [1]:
connections = []
with open('input_day_12.txt') as f:
    for line in f:
        connections.append(line.strip())

In [2]:
connections[-5:]

['1995 <-> 773, 1499',
 '1996 <-> 95, 1996',
 '1997 <-> 510, 796',
 '1998 <-> 626',
 '1999 <-> 964, 1568']

In [3]:
connections_dict = {}
def parse_connection(conn):
    prog, comlist = conn.split(' <-> ')
    prog = int(prog)
    comlist = [int(p) for p in comlist.split(',')]
    connections_dict[prog] = comlist
    
for conn in connections:
    parse_connection(conn)

In [4]:
def group_from_prog(prog):
    ctp = connections_dict[prog][:]
    to_visit = []
    visited = []
    to_visit.extend(ctp)
    while to_visit:
        pv = to_visit.pop()
        for p in connections_dict[pv]:
            if p not in visited:
                ctp.append(p)
                visited.append(p)
                to_visit.extend(connections_dict[p])
    return set(ctp)

In [5]:
len(group_from_prog(0))

113

In [6]:
all_progs = set(connections_dict.keys())
grouped_progs = set()
groups = 0
ungrouped_progs = all_progs.difference(grouped_progs)
while ungrouped_progs:
    group = group_from_prog(next(iter(ungrouped_progs)))
    grouped_progs.update(group)
    ungrouped_progs = all_progs.difference(grouped_progs)
    groups += 1
groups

202

# day 11

Struggled for a few minutes with coordinates for the hex grid, then googled and got the idea for the hex-as-3d from [here](http://keekerdc.com/2011/03/hexagon-grids-coordinate-systems-and-distance-calculations/). Obviously, it makes the solution fairly trivial.

In [1]:
with open('input_day_11.txt') as f:
    child_walk = f.read().strip().split(',')

In [2]:
compass_to_3d_offset = {'n': (-1, 1, 0),
                        'ne': (-1, 0, 1),
                        'se': (0, -1, 1),
                        's': (1, -1, 0),
                        'sw': (1, 0, -1),
                        'nw': (0, 1, -1)}

In [3]:
def add_vecs(va, vb):
    return tuple(map(lambda t: t[0] + t[1], zip(va, vb)))

In [4]:
cur_pos = (0, 0, 0)
max_dist = 0
for step in child_walk:
    cur_pos = add_vecs(cur_pos, compass_to_3d_offset[step])
    max_dist = max([max_dist] + list(map(abs, cur_pos)))
cur_pos, max(map(abs, cur_pos)), max_dist

((314, -759, 445), 759, 1501)

# day 10

In [1]:
with open('input_day_10.txt') as f:
    puzinp = f.read().strip()

In [2]:
numbers_max = 255
#numbers_max = 4
numbers = list(range(numbers_max + 1))
current_position = 0
skip_size = 0
overshot = False
lengths = [int(n) for n in puzinp.split(',')]
#lengths = [3, 4, 1, 5]

for length in lengths:
    #print(numbers)
    rev_span_start = current_position
    rev_span_end = rev_span_start + length
    if rev_span_end > numbers_max:
        overshot = True
        overshoot = rev_span_end - numbers_max - 1
        numbers = numbers + numbers
    numbers[rev_span_start:rev_span_end] = reversed(numbers[rev_span_start:rev_span_end])
    #print(list(reversed(numbers[rev_span_start:rev_span_end])))
    if overshot:
        numbers[0:overshoot] = numbers[numbers_max+1:numbers_max+1+overshoot]
        numbers = numbers[:numbers_max + 1]
        overshot = False
    current_position = (current_position + length + skip_size) % (numbers_max + 1)
    #print(f'position: {current_position}, skip size: {skip_size}')
    skip_size += 1
assert sorted(numbers) == list(range(256))
#print()

numbers[0]*numbers[1]

8536

In [3]:
part_two_input = [ord(ch) for ch in puzinp] + [17, 31, 73, 47, 23]

In [4]:
from functools import reduce
from operator import xor

In [5]:
numbers_max = 255
numbers = list(range(numbers_max + 1))
current_position = 0
skip_size = 0
overshot = False
lengths = part_two_input

for _ in range(64):
    for length in lengths:
        rev_span_start = current_position
        rev_span_end = rev_span_start + length
        if rev_span_end > numbers_max:
            overshot = True
            overshoot = rev_span_end - numbers_max - 1
            numbers = numbers + numbers
        numbers[rev_span_start:rev_span_end] = reversed(numbers[rev_span_start:rev_span_end])
        if overshot:
            numbers[0:overshoot] = numbers[numbers_max+1:numbers_max+1+overshoot]
            numbers = numbers[:numbers_max + 1]
            overshot = False
        current_position = (current_position + length + skip_size) % (numbers_max + 1)
        skip_size += 1
assert sorted(numbers) == list(range(256))

sparse_hashes = []
while numbers:
    take, numbers = numbers[:16], numbers[16:]
    sparse_hashes.append(reduce(xor, take))

''.join([hex(n)[2:] for n in sparse_hashes])

'aff593797989d665349efe11bb4fd99b'

# day 9

Didn't like the nesting of the first approach.

Tried combining conditions for the seconds approach. I think it is harder to read, even though less nested.

For the third approach, I:
- moved short / specific / non-'default' cases to earliest positions in if/elif stack
- elif then allowed coverage of remaining options more clearly, allowing nesting to be halved

I also tried seeing if commenting some of the later elif branches would add to readability, and my opinion was that it did not.

Ultimately, I find the third version to be a modest but useful improvement in readability vs the first approach.

In [1]:
with open('input_day_9.txt') as f:
    puzinp = f.read().strip()

In [2]:
len(puzinp)

24038

In [3]:
score = 0
nesting = 0
garb_ct = 0
in_garb = False
skip = False
for c in puzinp:
    if not skip:
        if c == '!':
            skip = True
        else:
            if not in_garb:
                if c == '<':
                    in_garb = True
                elif c == '{':
                    nesting += 1
                    score += nesting
                elif c == '}':
                    nesting -= 1
            else:
                if c == '>':
                    in_garb = False
                else:
                    garb_ct += 1
    else:
        skip = False    

score, garb_ct

(16689, 7982)

In [4]:
score = 0
nesting = 0
garb_ct = 0
in_garb = False
skip = False
for c in puzinp:
    if not skip and c == '!':
        skip = True
    elif not skip and not in_garb:
        if c == '<':
            in_garb = True
        elif c == '{':
            nesting += 1
            score += nesting
        elif c == '}':
            nesting -= 1
    elif not skip:  # and in_garb
        if c == '>':
            in_garb = False
        else:
            garb_ct += 1
    else:  # skip == True, so only unset
        skip = False    

score, garb_ct

(16689, 7982)

In [5]:
score = 0
nesting = 0
garb_ct = 0
in_garb = False
skip = False
for c in puzinp:
    if skip:
        skip = False
    elif c == '!':
        skip = True
    elif in_garb:
        if c == '>':
            in_garb = False
        else:
            garb_ct += 1 
    else:
        if c == '<':
            in_garb = True
        elif c == '{':
            nesting += 1
            score += nesting
        elif c == '}':
            nesting -= 1  

score, garb_ct

(16689, 7982)

# day 8

In [1]:
instructions = []
with open('input_day_8.txt') as f:
    for line in f:
        instructions.append(line.strip())

In [2]:
instructions[:10]

['o dec -427 if wnh < -1',
 'ugc dec -128 if baj <= 3',
 'ugc inc 294 if xml <= -1',
 'ugc inc -277 if wfu < -2',
 'l inc 302 if sqf == 0',
 'l inc -940 if wnh > -6',
 'vke inc 687 if fy >= -3',
 'b inc 976 if ugc > 118',
 'rz dec 201 if jyi < 5',
 'y dec -960 if l >= -647']

In [3]:
from collections import defaultdict

registers = defaultdict(int)
all_time_max = 0
for ins in instructions:
    reg, op, val, _, cond_reg, cond_rest = ins.split(maxsplit=5)
    if op == 'inc':
        op = '+='
    elif op == 'dec':
        op = '-='
    test_expression = "test = registers['" + cond_reg + "'] " + cond_rest
    action_expression = ' '.join(["registers['" + reg + "']", op, val])
    exec(test_expression)
    if test:
        exec(action_expression)
    all_time_max = max([all_time_max, max(registers.values())])

In [4]:
max(registers.values())

4448

In [5]:
all_time_max

6582

# day 7

In [1]:
reports = []
with open('input_day_7.txt') as f:
    for line in f:
        reports.append(line.strip())

In [2]:
for ri, r in enumerate(reports[:10]):
    print(ri, r)

0 xsddbi (61)
1 nqtowev (11)
2 xwohr (82)
3 flejt (36)
4 idwpug (54)
5 uoxzkp (51)
6 choeijs (54)
7 gmsjkn (65)
8 txszqu (687) -> mvjqmad, lwqlyjq, jlgnsu
9 zhlfdac (15)


In [3]:
from collections import defaultdict

In [4]:
held_sets = []
progs = defaultdict(dict)
for rep in reports:
    prog, *held = rep.replace(' -> ',',').split(',')
    name, weight = prog.split()
    weight = int(weight[1:-1])
    progs[name]['weight'] = weight
    held = [h.strip() for h in held]
    progs[name]['holds'] = held
    for h in held:
        progs[h]['on'] = name
    if held:
        held_sets.append(held)

In [5]:
for k, v in progs.items():
    if 'on' not in v.keys():
        print(k)

fbgguv


In [6]:
for k in list(progs.keys())[:10]:
    print(k, progs[k])

xsddbi {'weight': 61, 'holds': [], 'on': 'ptawnqq'}
nqtowev {'weight': 11, 'holds': [], 'on': 'nsxrvpg'}
xwohr {'weight': 82, 'holds': [], 'on': 'utyrttu'}
flejt {'weight': 36, 'holds': [], 'on': 'ydqgvnf'}
idwpug {'weight': 54, 'holds': [], 'on': 'vgqmfj'}
uoxzkp {'weight': 51, 'holds': [], 'on': 'ndhitxf'}
choeijs {'weight': 54, 'holds': [], 'on': 'thneux'}
gmsjkn {'weight': 65, 'holds': [], 'on': 'tmigcw'}
txszqu {'weight': 687, 'holds': ['mvjqmad', 'lwqlyjq', 'jlgnsu'], 'on': 'evows'}
mvjqmad {'on': 'txszqu', 'weight': 359, 'holds': ['swaxyxe', 'perbvgz']}


In [7]:
def get_tot_wt(prog):
    wt = progs[prog]['weight']
    for hp in progs[prog]['holds']:
        wt += get_tot_wt(hp)
    progs[prog]['tot_weight'] = wt
    return wt

In [8]:
get_tot_wt('fbgguv')

537824

In [9]:
held_sets[:5]

[['mvjqmad', 'lwqlyjq', 'jlgnsu'],
 ['nbrvl', 'bcmbao', 'vfimqtl'],
 ['molth', 'atqewc'],
 ['tyjyc', 'xyemll'],
 ['jxsmuey', 'uelnii', 'vcwezm', 'uxnwtp']]

In [10]:
for hs in held_sets:
    wts = [progs[h]['tot_weight'] for h in hs]
    if len(set(wts)) > 1:
        print(list(zip(hs, wts)))

[('bkomu', 89627), ('tynxlau', 89627), ('sfruur', 89632), ('zxvbb', 89627), ('khupkt', 89627), ('xkntkvz', 89627)]
[('twbxgv', 17913), ('evows', 17913), ('xwwzp', 17913), ('vsfgg', 17913), ('zsasjr', 17918)]
[('jopcvyb', 2548), ('pyoma', 2548), ('jdxfsa', 2553), ('liwlcz', 2548), ('cxetjr', 2548), ('vifbi', 2548), ('tlmxygb', 2548)]


In [11]:
[(h, progs[h]['tot_weight']) for h in progs['sfruur']['holds']]

[('twbxgv', 17913),
 ('evows', 17913),
 ('xwwzp', 17913),
 ('vsfgg', 17913),
 ('zsasjr', 17918)]

In [12]:
[(h, progs[h]['tot_weight']) for h in progs['zsasjr']['holds']]

[('jopcvyb', 2548),
 ('pyoma', 2548),
 ('jdxfsa', 2553),
 ('liwlcz', 2548),
 ('cxetjr', 2548),
 ('vifbi', 2548),
 ('tlmxygb', 2548)]

In [13]:
[(h, progs[h]['tot_weight']) for h in progs['jdxfsa']['holds']]

[('lugwl', 228), ('ckcjr', 228), ('umutl', 228)]

In [14]:
progs['jdxfsa']

{'holds': ['lugwl', 'ckcjr', 'umutl'],
 'on': 'zsasjr',
 'tot_weight': 2553,
 'weight': 1869}

# day 6

In [1]:
with open('input_day_6.txt') as f:
    mem_banks = f.read().strip().split()
    mem_banks = [int(b) for b in mem_banks]

seen_configs = {}
config_repeated = False
bank_ct = len(mem_banks)
redistribution_count = 0

while not config_repeated:
    dist_ct = max(mem_banks)
    bank_index = mem_banks.index(dist_ct)
    mem_banks[bank_index] = 0
    while dist_ct:
        bank_index += 1
        if bank_index == bank_ct:
            bank_index = 0
        mem_banks[bank_index] += 1
        dist_ct -= 1
    redistribution_count += 1
    if tuple(mem_banks) in seen_configs.keys():
        config_repeated = True
    else:
        seen_configs[tuple(mem_banks)] = redistribution_count
        
redistribution_count

5042

In [2]:
redistribution_count-seen_configs[tuple(mem_banks)]

1086

# day 5

In [1]:
jos = []
with open('input_day_5.txt') as f:
    for line in f:
        jos.append(int(line))

max_index = len(jos) - 1
index = 0
steps = 0
while 0 <= index <= max_index:
    jo = jos[index]
    jos[index] += 1 if jo < 3 else -1
    steps += 1
    index += jo

In [2]:
steps

24490906

# day 4

In [1]:
valid_count = 0
with open('input_day_4.txt') as f:
    for line in f:
        words = line.split()
        words = [''.join(sorted(list(w))) for w in words]
        if len(words) == len(set(words)):
            valid_count += 1
valid_count

167

# day 3

I watched Joel Grus’ livecoding solution of same, and learned two things:
1. Using a dictionary for storing the grid cells allows direct lookup and modification, and saves space vs. a fixed size grid (which is made larger than it needs to be in order to accommodate uncertainty in the final required size). The easier lookup is a very compelling reason to use this approach.
2. This was an opportunity to use generator functions (a modest number of yields, vs looping over three lists of indices.

In [2]:
puzzle_input = 277678

In [3]:
puzzle_input = 1024

In [4]:
import math


def next_square_of_odd(inp):
    trial = inp
    while math.sqrt(trial) % 2 != 1:
        trial += 1
    return int(math.sqrt(trial))

def day_3(inp):
    ns = next_square_of_odd(inp)
    te = (ns - 1)//2
    fo = (ns**2-inp) % (ns - 1)
    print(ns**2, ns, te, fo, abs(fo-te))
    return te + abs(fo-te)

#assert day_3(1) == 0
assert day_3(12) == 3
assert day_3(23) == 2
assert day_3(1024) == 31

25 5 2 1 1
25 5 2 2 0
1089 33 16 1 15


In [5]:
day_3(22)

25 5 2 3 1


3

In [6]:
day_3(23)

25 5 2 2 0


2

In [7]:
day_3(1024)

1089 33 16 1 15


31

In [8]:
day_3(277678)

277729 527 263 51 212


475

In [9]:
def solution(num):
    width = 1001
    solarr = [[0] * width] * width
    cr = (width - 1) // 2
    cc = (width - 1) // 2
    
    def set_cell_in_solarr(r, c, v):
        row = solarr[r][:]
        row[c] = v
        solarr[r] = row
    
    def sum_around(center_row, center_col):
        sa = sum(solarr[center_row - 1][center_col - 1:center_col + 2])
        sa += sum(solarr[center_row ][center_col - 1:center_col + 2])
        sa += sum(solarr[center_row + 1][center_col - 1:center_col + 2])
        return sa

    set_cell_in_solarr(cr, cc, 1)
    loop_num = 0
    found = False
    while not found:
        loop_num += 1
        edge_list = range(2*loop_num)
        cr += 1
        cc += 1
        # loop proceeds up, left, down, right, loop_num + 2 steps each time
        for rc_step in ((-1, 0), (0, -1), (1, 0), (0, 1)):
            for _ in edge_list:
                if not found:
                    cr, cc = cr + rc_step[0], cc + rc_step[1]
                    csum = sum_around(cr, cc)
                    if csum > num:
                        found = True
                    set_cell_in_solarr(cr, cc, csum)
    for r in solarr[496:506]:
        print(r[496:506])
    return csum

In [10]:
puzzle_input = 277678
solution(puzzle_input)

[0, 0, 0, 0, 0, 0, 279138, 266330, 130654, 0]
[0, 6591, 6444, 6155, 5733, 5336, 5022, 2450, 128204, 0]
[0, 13486, 147, 142, 133, 122, 59, 2391, 123363, 0]
[0, 14267, 304, 5, 4, 2, 57, 2275, 116247, 0]
[0, 15252, 330, 10, 1, 1, 54, 2105, 109476, 0]
[0, 16295, 351, 11, 23, 25, 26, 1968, 103128, 0]
[0, 17008, 362, 747, 806, 880, 931, 957, 98098, 0]
[0, 17370, 35487, 37402, 39835, 42452, 45220, 47108, 48065, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


279138

In [11]:
solution(142)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 147, 142, 133, 122, 59, 0, 0, 0]
[0, 0, 0, 5, 4, 2, 57, 0, 0, 0]
[0, 0, 0, 10, 1, 1, 54, 0, 0, 0]
[0, 0, 0, 11, 23, 25, 26, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


147

# day 2

In [1]:
def ev_div(data):
    for a in data:
        for b in data:
            lrg = max([a, b])
            sm = min([a, b])
            if lrg != sm and lrg % sm == 0:
                return lrg//sm

In [2]:
spreadsheet_sum = 0
ev_div_sum = 0
with open('input_day_2.txt') as f:
    for line in f:
        line_dat = [int(el) for el in line.strip().split('\t')]
        spreadsheet_sum += max(line_dat) - min(line_dat)
        div = ev_div(line_dat)
        ev_div_sum += div
        
spreadsheet_sum, ev_div_sum

(45158, 294)

# day 1

In [1]:
puzzle_input = '111831362354551173134957758417849716877188716338227121869992652972154651632296676464285261171625892888598738721925357479249486886375279741651224686642647267979445939836673253446489428761486828844713816198414852769942459766921928735591892723619845983117283575762694758223956262583556675379533479458964152461973321432768858165818549484229241869657725166769662249574889435227698271439423511175653875622976121749344756734658248245212273242115488961818719828258936653236351924292251821352389471971641957941593141159982696396228218461855752555358856127582128823657548151545741663495182446281491763249374581774426225822474112338745629194213976328762985884127324443984163571711941113986826168921187567861288268744663142867866165546795621466134333541274633769865956692539151971953651886381195877638919355216642731848659649263217258599456646635412623461138792945854536154976732167439355548965778313264824237176152196614333748919711422188148687299757751955297978137561935963366682742334867854892581388263132968999722366495346854828316842352829827989419393594846893842746149235681921951476132585199265366836257322121681471877187847219712325933714149151568922456111149524629995933156924418468567649494728828858254296824372929211977446729691143995333874752448315632185286348657293395339475256796591968717487615896959976413637422536563273537972841783386358764761364989261322293887361558128521915542454126546182855197637753115352541578972298715522386683914777967729562229395936593272269661295295223113186683594678533511783187422193626234573849881185849626389774394351115527451886962844431947188429195191724662982411619815811652741733744864411666766133951954595344837179635668177845937578575117168875754181523584442699384167111317875138179567939174589917894597492816476662186746837552978671142265114426813792549412632291424594239391853358914643327549192165466628737614581458189732579814919468795493415762517372227862614224911844744711698557324454211123571327224554259626961741919243229688684838813912553397698937237114287944446722919198743189848428399356842626198635297851274879128322358195585284984366515428245928111112613638341345371'

In [2]:
pi_rot = puzzle_input[1:] + puzzle_input[0]

In [3]:
sum([int(a) for a, b in zip(puzzle_input, pi_rot) if a == b])

1044

In [4]:
rot = len(puzzle_input)//2

In [5]:
pi_rot = puzzle_input[rot:] + puzzle_input[:rot]

In [6]:
sum([int(a) for a, b in zip(puzzle_input, pi_rot) if a == b])

1054